# Open nested combat: TOP and StrokeMRI combined (topmri) and SABRE open nested-harmonized datasets

Note this must be run in the `opnc` environment

## import libraries

In [ ]:
import os
import sys
#from itertools import permutations

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets


from sklearn.preprocessing import LabelEncoder


sys.path.insert(0, '../../') # path to cvasl functions
import cvasl.seperated as sep
import cvasl.harmony as har

## Read in our data

In [ ]:
# EDIS_path = '../our_datasets/EDIS/'
# HELIUS_path = '../our_datasets/HELIUS/'
# Insight46_path = '../our_datasets/Insight46/'
SABRE_path = '../our_datasets/SABRE/'
MRI_path = '../our_datasets/StrokeMRI/'
TOP_path = '../our_datasets/TOP/'
file_name = 'TrainingDataComplete.csv'

TOP_file = os.path.join(TOP_path, file_name)
MRI_file = os.path.join(MRI_path, file_name)
#EDIS_file = os.path.join(EDIS_path, file_name)
#HELIUS_file = os.path.join(HELIUS_path, file_name)
#Insight46_file = os.path.join(Insight46_path, file_name)
SABRE_file = os.path.join(SABRE_path, file_name)

# EDIS = pd.read_csv(EDIS_file)
# #HELIUS = pd.read_csv(HELIUS_file)
# Insight46 = pd.read_csv(Insight46_file)
# SABRE = pd.read_csv(SABRE_file)
TOP = pd.read_csv(TOP_file)
MRI = pd.read_csv(MRI_file)
SABRE = pd.read_csv(SABRE_file)

In [ ]:
TOP.head(3)

In [ ]:
MRI.tail(3)

In [ ]:
topmri = pd.concat([MRI ,TOP])
topmri.head(3)

In [ ]:
SABRE.tail(3)

In [ ]:
topmri.shape

In [ ]:
topmri.columns = topmri.columns.str.lower()
topmri = topmri.drop(['site', 'id'], axis= 1)
topmri.head(3)

In [ ]:
SABRE.columns = SABRE.columns.str.lower()
SABRE = SABRE.drop(['site', 'id'], axis= 1)
SABRE.tail(3)

In [ ]:
topmri_batch_testing_df = topmri[['participant_id','age', 'sex']]
topmri_batch_testing_df['site'] = 0
column_to_move = topmri_batch_testing_df.pop("site")
topmri_batch_testing_df.insert(1, "site", column_to_move)
topmri_batch_testing_df.head(3)

In [ ]:
sabre_batch_testing_df = SABRE[['participant_id','age', 'sex']]
sabre_batch_testing_df['site'] = 1
column_to_move = sabre_batch_testing_df.pop("site")
sabre_batch_testing_df.insert(1, "site", column_to_move)
sabre_batch_testing_df.head(3)

In [ ]:
batch_testing_df = pd.concat([topmri_batch_testing_df, sabre_batch_testing_df], ignore_index=True)
#batch_testing_df = sep.recode_sex_to_numeric(batch_testing_df)
batch_testing_df.tail(5)

In [ ]:
# Loading in batch effects
batch_testing_list = ['site']
# Loading in clinical covariates
categorical_testing_cols = ['sex']
continuous_testing_cols = ['age']

In [ ]:
# ASL
data_testing_df = pd.concat([topmri,SABRE])
#data_testing_df = # ASL
data_testing_df = data_testing_df.drop(columns=['age','sex'])  
data_testing_df.tail(3)

In [ ]:
data_testing_df = data_testing_df.reset_index(drop=True)
data_testing_df = data_testing_df.dropna()
data_testing_df = data_testing_df.merge(batch_testing_df['participant_id'], 
                                        left_on='participant_id', right_on='participant_id')
dat_testing = data_testing_df.iloc[:, 1:]
dat_testing = dat_testing.T.apply(pd.to_numeric)
caseno_testing = data_testing_df['participant_id']
covars_testing = batch_testing_df.drop('participant_id',1)

In [ ]:
data_testing_df.columns

In [ ]:
# Merging batch effects, clinical covariates
covars_testing_string = pd.DataFrame()
covars_testing_string[categorical_testing_cols] = covars_testing[categorical_testing_cols].copy()
covars_testing_quant = covars_testing[continuous_testing_cols]
#covars_testing_quant

In [ ]:
# Encoding categorical variables
covars_testing_cat = pd.DataFrame()
for col_testing in covars_testing_string:
    stringcol_testing = covars_testing_string[col_testing]
    le = LabelEncoder()
    le.fit(list(stringcol_testing))
    covars_testing_cat[col_testing] = le.transform(stringcol_testing)
#covars_testing_cat

In [ ]:
covars_testing_final = pd.concat([covars_testing_cat, covars_testing_quant], axis=1)
#covars_testing_final

In [ ]:
# # # FOR GMM COMBAT VARIANTS:
# # # Adding GMM Split to batch effects
filepath2 = 'Testing/OPPNComBat/ResultTesting'
if not os.path.exists(filepath2):
    os.makedirs(filepath2)
gmm_testing_df = har.GMMSplit(dat_testing, caseno_testing, filepath2)

In [ ]:
batch_testing_df.head(2)

In [ ]:
gmm_testing_df.head(2)

In [ ]:
gmm_testing_df_merge = batch_testing_df.merge(gmm_testing_df, right_on='Patient', left_on='participant_id')
gmm_testing_df_merge['GMM'] = gmm_testing_df_merge['Grouping'] 
gmm_testing_df_merge

In [ ]:
covars_testing_final = gmm_testing_df_merge.drop(['participant_id','Patient','Grouping'],1)
categorical_testing_cols = categorical_testing_cols + ['GMM']

In [ ]:
output_testing_df = har.OPNestedComBat(dat_testing,
                                   covars_testing_final,
                                   batch_testing_list,
                                   filepath2, categorical_cols=categorical_testing_cols,
                                  continuous_cols=continuous_testing_cols)

In [ ]:
write_testing_df = pd.concat([caseno_testing, output_testing_df], axis=1) 
write_testing_df.to_csv(filepath2+'/Mfeatures_testing_NestedComBat.csv') # write results fo file
dat_testing_input = dat_testing.transpose()
dat_testing_input.to_csv(filepath2+'/Mfeatures_input_testing_NestedComBat.csv')
covars_testing_final.to_csv(filepath2+'/Mcovars_input_testing_NestedComBat.csv')

In [ ]:
# write harmonized dataset 
complete_harmonised = pd.concat([write_testing_df, covars_testing_final], axis=1) 
complete_harmonised.head(3)

In [ ]:
complete_harmonised.columns

# split dataframe back up into parts for running
## from complete_harmonised

In [ ]:
sabre_opn_harmonized = complete_harmonised[complete_harmonised['site'] == 1]
sabre_opn_harmonized = sabre_opn_harmonized.drop(columns=['site', 'GMM',])
sabre_opn_harmonized.head(3)

In [ ]:
topmri_opn_harmonized = complete_harmonised[complete_harmonised['site'] == 0]
topmri_opn_harmonized = topmri_opn_harmonized.drop(columns=['site', 'GMM',])
#top_opn_harmonized.head(3)

In [ ]:
topmri_opn_harmonized.to_csv('harm_results/open_nested_combat/topmri_opn_harmonized_to_sab.csv')
sabre_opn_harmonized.to_csv('harm_results/open_nested_combat/sabre_opn_harmonized.csv')

## further investigation and graphing

In [ ]:
sabre_opn_harmonized.head(3)

In [ ]:
SABRE.columns

In [ ]:
sabre_opn_harmonized.columns

In [ ]:
complete_harmonised.columns

In [ ]:
complete_nonharmonised = pd.concat([caseno_testing, dat_testing_input], axis=1) 
complete_nonharmonised = pd.concat([complete_nonharmonised, covars_testing_final], axis=1) 
complete_nonharmonised.columns

In [ ]:
complete_harmonised['harmonization'] = 'H'

complete_nonharmonised['harmonization'] = 'UH'

In [ ]:
feature_list = ['gm_vol', 'wm_vol', 'csf_vol',
       'gm_icvratio', 'gmwm_icvratio', 'wmhvol_wmvol', 'wmh_count',
       'deepwm_b_cov', 'aca_b_cov', 'mca_b_cov', 'pca_b_cov', 'totalgm_b_cov',
       'deepwm_b_cbf', 'aca_b_cbf', 'mca_b_cbf', 'pca_b_cbf', 'totalgm_b_cbf']
har.compare_harm_multi_site_violins(
        complete_nonharmonised,
        complete_harmonised,
        feature_list
)

<!-- # split dataframe back up into parts for running
## from complete_harmonised -->